In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.preprocessing import LabelBinarizer, StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
train = pd.read_csv('./data/rawdata/application_train.csv.zip',compression='zip')
test = pd.read_csv('./data/rawdata/application_test.csv.zip',compression='zip')

In [3]:
train.shape
train.head()
train.tail()
test.shape
test.tail()

(307511, 122)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
307510,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,1.0


(48744, 121)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
48743,456250,Cash loans,F,Y,N,0,135000.0,312768.0,24709.5,270000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0


In [106]:
# train.TARGET.groupby(train.AMT_REQ_CREDIT_BUREAU_YEAR).agg(['mean','count'])
# train.AMT_REQ_CREDIT_BUREAU_YEAR.value_counts(dropna=False)
# [1 if b == 'Family' else 0 for b in ['Unaccompanied', 'Family', 'Spouse, partner', 'Children', 'Other_A','Other_B', 'Group of people']]

# import category_encoders as ce
# encoder = ce.OneHotEncoder(cols=['FLAG_DOCUMENT_5','FLAG_OWN_CAR'])
# encoder.fit(train[['FLAG_DOCUMENT_5','FLAG_OWN_CAR']])
# encoder.transform(train[['FLAG_DOCUMENT_5','FLAG_OWN_CAR']])

In [4]:
feature_dict = {'NAME_CONTRACT_TYPE':'category_features', 'CODE_GENDER':'category_features', 'FLAG_OWN_CAR':'boolean_features', 'FLAG_OWN_REALTY':'boolean_features', 'CNT_CHILDREN':'count_features', 'AMT_INCOME_TOTAL':'amount_feautres', 'AMT_CREDIT':'amount_feautres', 'AMT_ANNUITY':'amount_feautres', 'AMT_GOODS_PRICE':'amount_feautres', 'NAME_TYPE_SUITE':'category_features', 'NAME_INCOME_TYPE':'category_features', 'NAME_EDUCATION_TYPE':'category_features', 'NAME_FAMILY_STATUS':'category_features', 'NAME_HOUSING_TYPE':'category_features', 'REGION_POPULATION_RELATIVE':'normal_features', 'DAYS_BIRTH':'amount_feautres', 'DAYS_EMPLOYED':'amount_feautres', 'DAYS_REGISTRATION':'amount_feautres', 'DAYS_ID_PUBLISH':'amount_feautres', 'OWN_CAR_AGE':'amount_feautres', 'FLAG_MOBIL':'boolean_features', 'FLAG_EMP_PHONE':'boolean_features', 'FLAG_WORK_PHONE':'boolean_features', 'FLAG_CONT_MOBILE':'boolean_features', 'FLAG_PHONE':'boolean_features', 'FLAG_EMAIL':'boolean_features', 'OCCUPATION_TYPE':'category_features', 'CNT_FAM_MEMBERS':'count_features', 'REGION_RATING_CLIENT':'category_features', 'REGION_RATING_CLIENT_W_CITY':'category_features', 'WEEKDAY_APPR_PROCESS_START':'category_features', 'HOUR_APPR_PROCESS_START':'category_features', 'REG_REGION_NOT_LIVE_REGION':'boolean_features', 'REG_REGION_NOT_WORK_REGION':'boolean_features', 'LIVE_REGION_NOT_WORK_REGION':'boolean_features', 'REG_CITY_NOT_LIVE_CITY':'boolean_features', 'REG_CITY_NOT_WORK_CITY':'boolean_features', 'LIVE_CITY_NOT_WORK_CITY':'boolean_features', 'ORGANIZATION_TYPE':'category_features', 'EXT_SOURCE_1':'normal_features', 'EXT_SOURCE_2':'normal_features', 'EXT_SOURCE_3':'normal_features', 'APARTMENTS_AVG':'normal_features', 'BASEMENTAREA_AVG':'normal_features', 'YEARS_BEGINEXPLUATATION_AVG':'normal_features', 'YEARS_BUILD_AVG':'normal_features', 'COMMONAREA_AVG':'normal_features', 'ELEVATORS_AVG':'normal_features', 'ENTRANCES_AVG':'normal_features', 'FLOORSMAX_AVG':'normal_features', 'FLOORSMIN_AVG':'normal_features', 'LANDAREA_AVG':'normal_features', 'LIVINGAPARTMENTS_AVG':'normal_features', 'LIVINGAREA_AVG':'normal_features', 'NONLIVINGAPARTMENTS_AVG':'normal_features', 'NONLIVINGAREA_AVG':'normal_features', 'APARTMENTS_MODE':'normal_features', 'BASEMENTAREA_MODE':'normal_features', 'YEARS_BEGINEXPLUATATION_MODE':'normal_features', 'YEARS_BUILD_MODE':'normal_features', 'COMMONAREA_MODE':'normal_features', 'ELEVATORS_MODE':'normal_features', 'ENTRANCES_MODE':'normal_features', 'FLOORSMAX_MODE':'normal_features', 'FLOORSMIN_MODE':'normal_features', 'LANDAREA_MODE':'normal_features', 'LIVINGAPARTMENTS_MODE':'normal_features', 'LIVINGAREA_MODE':'normal_features', 'NONLIVINGAPARTMENTS_MODE':'normal_features', 'NONLIVINGAREA_MODE':'normal_features', 'APARTMENTS_MEDI':'normal_features', 'BASEMENTAREA_MEDI':'normal_features', 'YEARS_BEGINEXPLUATATION_MEDI':'normal_features', 'YEARS_BUILD_MEDI':'normal_features', 'COMMONAREA_MEDI':'normal_features', 'ELEVATORS_MEDI':'normal_features', 'ENTRANCES_MEDI':'normal_features', 'FLOORSMAX_MEDI':'normal_features', 'FLOORSMIN_MEDI':'normal_features', 'LANDAREA_MEDI':'normal_features', 'LIVINGAPARTMENTS_MEDI':'normal_features', 'LIVINGAREA_MEDI':'normal_features', 'NONLIVINGAPARTMENTS_MEDI':'normal_features', 'NONLIVINGAREA_MEDI':'normal_features', 'FONDKAPREMONT_MODE':'category_features', 'HOUSETYPE_MODE':'category_features', 'TOTALAREA_MODE':'normal_features', 'WALLSMATERIAL_MODE':'category_features', 'EMERGENCYSTATE_MODE':'category_features', 'OBS_30_CNT_SOCIAL_CIRCLE':'count_features', 'DEF_30_CNT_SOCIAL_CIRCLE':'count_features', 'OBS_60_CNT_SOCIAL_CIRCLE':'count_features', 'DEF_60_CNT_SOCIAL_CIRCLE':'count_features', 'DAYS_LAST_PHONE_CHANGE':'count_features', 'FLAG_DOCUMENT_2':'boolean_features', 'FLAG_DOCUMENT_3':'boolean_features', 'FLAG_DOCUMENT_4':'boolean_features', 'FLAG_DOCUMENT_5':'boolean_features', 'FLAG_DOCUMENT_6':'boolean_features', 'FLAG_DOCUMENT_7':'boolean_features', 'FLAG_DOCUMENT_8':'boolean_features', 'FLAG_DOCUMENT_9':'boolean_features', 'FLAG_DOCUMENT_10':'boolean_features', 'FLAG_DOCUMENT_11':'boolean_features', 'FLAG_DOCUMENT_12':'boolean_features', 'FLAG_DOCUMENT_13':'boolean_features', 'FLAG_DOCUMENT_14':'boolean_features', 'FLAG_DOCUMENT_15':'boolean_features', 'FLAG_DOCUMENT_16':'boolean_features', 'FLAG_DOCUMENT_17':'boolean_features', 'FLAG_DOCUMENT_18':'boolean_features', 'FLAG_DOCUMENT_19':'boolean_features', 'FLAG_DOCUMENT_20':'boolean_features', 'FLAG_DOCUMENT_21':'boolean_features', 'AMT_REQ_CREDIT_BUREAU_HOUR':'count_features', 'AMT_REQ_CREDIT_BUREAU_DAY':'count_features', 'AMT_REQ_CREDIT_BUREAU_WEEK':'count_features', 'AMT_REQ_CREDIT_BUREAU_MON':'count_features', 'AMT_REQ_CREDIT_BUREAU_QRT':'count_features', 'AMT_REQ_CREDIT_BUREAU_YEAR':'count_features'}

In [5]:
category_features = [f for f in feature_dict if feature_dict[f] == 'category_features' ]
boolean_features = [f for f in feature_dict if feature_dict[f] == 'boolean_features' ]
amount_feautres = [f for f in feature_dict if feature_dict[f] == 'amount_feautres' ]
normal_features = [f for f in feature_dict if feature_dict[f] == 'normal_features' ]
count_features = [f for f in feature_dict if feature_dict[f] == 'count_features' ]

In [6]:

minmax = MinMaxScaler().fit(pd.concat([train[amount_feautres+count_features],test[amount_feautres+count_features]],axis = 0).fillna(0))

train[amount_feautres+count_features] = pd.DataFrame(minmax.transform(train[amount_feautres+count_features].fillna(0)), columns = amount_feautres+count_features)
test[amount_feautres+count_features] = pd.DataFrame(minmax.transform(test[amount_feautres+count_features].fillna(0)), columns = amount_feautres+count_features)


In [9]:
# bin
bins = 5
for rate in normal_features+amount_feautres+count_features:
    qcut = pd.qcut(train[rate],bins,retbins=True,duplicates='drop')[1]
    if len(qcut) <=2:
        continue
    qcut[0] = 0
    qcut[-1] = 1
    train[rate +'_bin'] = [rate + '_' + str(np.where(qcut>= r)[0][0]) if not np.isnan(r) else 'miss' for r in train[rate]]
    test[rate +'_bin'] =  [rate + '_' +  str(np.where(qcut>= r)[0][0]) if not np.isnan(r) else 'miss' for r in test[rate]]
    category_features.append(rate +'_bin')



In [10]:
taget_encoder = ce.TargetEncoder(cols=category_features + boolean_features)
taget_encoder.fit(train[category_features + boolean_features], train.TARGET)

onehot_encoder = ce.OneHotEncoder(cols = category_features + boolean_features)
onehot_encoder.fit(train[category_features + boolean_features])

ordinal_encoder = ce.OrdinalEncoder(cols = category_features)
ordinal_encoder.fit(train[category_features])

X_train = pd.concat([
    train[normal_features+amount_feautres+count_features].fillna(0),
    taget_encoder.transform(train[category_features + boolean_features]),
    onehot_encoder.transform(train[category_features + boolean_features]),
    ordinal_encoder.transform(train[category_features]),
], axis = 1)
y_train = train.TARGET


X_test = pd.concat([
    test[normal_features+amount_feautres+count_features].fillna(0),
    taget_encoder.transform(test[category_features + boolean_features]),
    onehot_encoder.transform(test[category_features + boolean_features]),
    ordinal_encoder.transform(test[category_features]),
], axis = 1)

TargetEncoder(cols=['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'ORGANIZATION_TYPE', ... 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21'],
       drop_invariant=False, handle_unknown='impute', impute_missing=True,
       min_samples_leaf=1, return_df=True, smoothing=1, verbose=0)

OneHotEncoder(cols=['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'ORGANIZATION_TYPE', ... 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21'],
       drop_invariant=False, handle_unknown='impute', impute_missing=True,
       return_df=True, verbose=0)

OrdinalEncoder(cols=['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'ORGANIZATION_TYPE', ...in', 'OBS_60_CNT_SOCIAL_CIRCLE_bin', 'DAYS_LAST_PHONE_CHANGE_bin', 'AMT_REQ_CREDIT_BUREAU_YEAR_bin'],
        drop_invariant=False, handle_unknown='impute', impute_missing=True,
        mapping=[{'col': 'NAME_CONTRACT_TYPE', 'mapping': [('Cash loans', 0), ('Revolving loans', 1)]}, {'col': 'CODE_GENDER', 'mapping': [('M', 0), ('F', 1), ('XNA', 2)]}, {'col': 'NAME_TYPE_SUITE', 'mapping': [('Unaccompanied', 0), ('Family', 1), ('Spouse, partner', 2), ('Children', 3), ('Other_A', 4), (n...EDIT_BUREAU_YEAR_2', 2), ('AMT_REQ_CREDIT_BUREAU_YEAR_4', 3), ('AMT_REQ_CREDIT_BUREAU_YEAR_3', 4)]}],
        return_df=True, verbose=0)

In [11]:
X_train.shape
X_train.head()

(307511, 988)

,REGION_POPULATION_RELATIVE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,...,DAYS_EMPLOYED_bin,DAYS_REGISTRATION_bin,DAYS_ID_PUBLISH_bin,OWN_CAR_AGE_bin,CNT_CHILDREN_bin,CNT_FAM_MEMBERS_bin,OBS_30_CNT_SOCIAL_CIRCLE_bin,OBS_60_CNT_SOCIAL_CIRCLE_bin,DAYS_LAST_PHONE_CHANGE_bin,AMT_REQ_CREDIT_BUREAU_YEAR_bin
0,0.018801,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,...,0,0,0,0,0,0,0,0,0,0
1,0.003541,0.311267,0.622246,0.000000,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,...,1,1,1,0,0,1,1,1,1,1
2,0.010032,0.000000,0.555912,0.729567,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,...,0,0,0,1,0,0,2,2,1,1
3,0.008019,0.000000,0.650442,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,...,2,2,0,0,0,1,0,0,1,1
4,0.028663,0.000000,0.322738,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,...,2,0,2,0,0,0,2,2,0,1


In [13]:
lr = LogisticRegression(C = 0.1, penalty = 'l1', random_state = 42)
lr.fit(X_train, y_train)
roc_auc_score(y_train, lr.predict_proba(X_train)[:,1])

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

0.75346654929739287

In [14]:
test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,DAYS_EMPLOYED_bin,DAYS_REGISTRATION_bin,DAYS_ID_PUBLISH_bin,OWN_CAR_AGE_bin,CNT_CHILDREN_bin,CNT_FAM_MEMBERS_bin,OBS_30_CNT_SOCIAL_CIRCLE_bin,OBS_60_CNT_SOCIAL_CIRCLE_bin,DAYS_LAST_PHONE_CHANGE_bin,AMT_REQ_CREDIT_BUREAU_YEAR_bin
0,100001,Cash loans,F,N,Y,0.00,0.000935,0.130787,0.079684,0.111111,...,DAYS_EMPLOYED_2,DAYS_REGISTRATION_3,DAYS_ID_PUBLISH_5,OWN_CAR_AGE_0,CNT_CHILDREN_0,CNT_FAM_MEMBERS_2,OBS_30_CNT_SOCIAL_CIRCLE_0,OBS_60_CNT_SOCIAL_CIRCLE_0,DAYS_LAST_PHONE_CHANGE_1,AMT_REQ_CREDIT_BUREAU_YEAR_0
1,100005,Cash loans,M,N,Y,0.00,0.000627,0.044387,0.067319,0.044444,...,DAYS_EMPLOYED_1,DAYS_REGISTRATION_1,DAYS_ID_PUBLISH_4,OWN_CAR_AGE_0,CNT_CHILDREN_0,CNT_FAM_MEMBERS_2,OBS_30_CNT_SOCIAL_CIRCLE_0,OBS_60_CNT_SOCIAL_CIRCLE_0,DAYS_LAST_PHONE_CHANGE_5,AMT_REQ_CREDIT_BUREAU_YEAR_3
2,100013,Cash loans,M,Y,Y,0.00,0.001512,0.154373,0.270427,0.155556,...,DAYS_EMPLOYED_1,DAYS_REGISTRATION_4,DAYS_ID_PUBLISH_3,OWN_CAR_AGE_1,CNT_CHILDREN_0,CNT_FAM_MEMBERS_2,OBS_30_CNT_SOCIAL_CIRCLE_0,OBS_60_CNT_SOCIAL_CIRCLE_0,DAYS_LAST_PHONE_CHANGE_3,AMT_REQ_CREDIT_BUREAU_YEAR_4
3,100028,Cash loans,F,N,Y,0.10,0.002474,0.382022,0.189975,0.388889,...,DAYS_EMPLOYED_2,DAYS_REGISTRATION_4,DAYS_ID_PUBLISH_2,OWN_CAR_AGE_0,CNT_CHILDREN_2,CNT_FAM_MEMBERS_4,OBS_30_CNT_SOCIAL_CIRCLE_0,OBS_60_CNT_SOCIAL_CIRCLE_0,DAYS_LAST_PHONE_CHANGE_1,AMT_REQ_CREDIT_BUREAU_YEAR_3
4,100038,Cash loans,M,Y,N,0.05,0.001320,0.144944,0.124278,0.154444,...,DAYS_EMPLOYED_2,DAYS_REGISTRATION_3,DAYS_ID_PUBLISH_2,OWN_CAR_AGE_2,CNT_CHILDREN_1,CNT_FAM_MEMBERS_3,OBS_30_CNT_SOCIAL_CIRCLE_0,OBS_60_CNT_SOCIAL_CIRCLE_0,DAYS_LAST_PHONE_CHANGE_3,AMT_REQ_CREDIT_BUREAU_YEAR_0


In [15]:
sub = pd.concat([
    test.SK_ID_CURR,
    pd.DataFrame({'TARGET':lr.predict_proba(X_test)[:,1]})
], axis = 1)


In [17]:
sub.head()
sub.to_csv('./data/submission/seconde_20180522.csv', index = False)

,SK_ID_CURR,TARGET
0,100001,0.047133
1,100005,0.104216
2,100013,0.028118
3,100028,0.028738
4,100038,0.171512


In [12]:
pd.concat([X_train,pd.DataFrame(y_train, columns = ['TARGET'])], axis = 1).to_csv('./data/input/train.csv', index = False)
# y_train.to_csv('./data/input/y_train.csv', index = False)
X_test.to_csv('./data/input/test.csv', index = False)
